In [4]:
# Get the data for expeirment
import pandas as pd 
import numpy as np
import time
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, RBF
import seaborn as sns
import TsModel
import GprModel
import TsModel_constant
import torch
from sklearn.metrics import r2_score, root_mean_squared_error
import matplotlib.pyplot as plt
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras
from keras import layers, models

device = torch.device("cuda:0")
print(f"Keras version is {keras.__version__}")
print(f"Num GPUs Available: {torch.cuda.device_count()}")
plt.rcParams.update({'font.size': 20})

Keras version is 3.1.1
Num GPUs Available: 1


In [16]:
exp32_results = {
    'gp_tsl_train_accuracy':gp_tsl_train_accuracy,
    'gp_tsl_test_accuracy':gp_tsl_test_accuracy,
    'gp_tsl_r2s':gp_tsl_r2s,
    'gp_tsl_time':gp_tsl_time,
    'gp_tsc_train_accuracy':gp_tsc_train_accuracy,
    'gp_tsc_test_accuracy':gp_tsc_test_accuracy,
    'gp_tsc_r2s':gp_tsc_r2s,
    'gp_tsc_time':gp_tsc_time,
    'gp_nn_tsl_train_accuracy':gp_nn_tsl_train_accuracy,
    'gp_nn_tsl_test_accuracy':gp_nn_tsl_test_accuracy,
    'gp_nn_tsl_r2s':gp_nn_tsl_r2s,
    'nn_tsl_r2s':nn_tsl_r2s,
    'gp_nn_tsl_time':gp_nn_tsl_time,
}

In [17]:
#%store exp32_results 

Stored 'exp32_results' (dict)


In [5]:
# Train and test the solar data set
x_train = pd.read_csv('Data/FES/solar_x_train.csv')
x_test = pd.read_csv('Data/FES/solar_x_test.csv')
y_train = pd.read_csv('Data/FES/solar_y_train.csv')
y_test = pd.read_csv('Data/FES/solar_y_test.csv')

In [6]:
number_of_folds = 10

# Source: From the TSmodel
def preprocess_data(train_x, test_x, train_y, test_y):
    feature_scaler = StandardScaler()
    scaled_train_x = feature_scaler.fit_transform(train_x)
    scaled_test_x = feature_scaler.transform(test_x)

    # target_scaler = StandardScaler()  
    # scaled_train_y = target_scaler.fit_transform(train_y)
    # scaled_test_y = target_scaler.transform(test_y)    
    # return scaled_train_x, scaled_test_x, scaled_train_y, scaled_test_y, feature_scaler, target_scaler    

    return scaled_train_x, scaled_test_x, feature_scaler    

def build_neural_network_model():
    model = models.Sequential()
    model.add(layers.Dense(10, activation=keras.activations.tanh))    
    model.add(layers.Dense(10, activation=keras.activations.tanh))    
    model.add(layers.Dense(1, activation=keras.activations.relu))
    return model

In [7]:
x_train, x_test, feature_scaler = preprocess_data(x_train, x_test, y_train, y_test)
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [8]:
nn_model = keras.saving.load_model('solar_nn.keras')

In [9]:
# TS model
# Linear conclusion

def ts_relu_wrapper(pred):
    return np.maximum(0, pred)


def get_tsl_model(x_train, y_train):
    tsl_model = TsModel.TsModel(number_of_rules=25, fuzzification_coefficient=1.2)
    tsl_model.fit(x_train, y_train)
    return tsl_model
# TS with linear conclusion
tsl_model = TsModel.TsModel(number_of_rules=25, fuzzification_coefficient=1.2)

start_time = time.time()
tsl_model.fit(x_train, y_train)
time_used = time.time() - start_time

FCM training RMSE: 267.1565067666455


In [10]:
# TS model
# Constant conclusion


tsc_model = TsModel_constant.TsModel_constant(number_of_rules=25, fuzzification_coefficient=1.2)

def get_tsc_model(x_train, y_train):
    tsc_model = TsModel_constant.TsModel_constant(number_of_rules=25, fuzzification_coefficient=1.2)
    tsc_model.fit(x_train, y_train)
    return tsc_model

start_time = time.time()
tsc_model.fit(x_train, y_train)
time_used = time.time() - start_time

(2207, 150)
[[2.29605688e-10 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.05665250e-16 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.55945023e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.02811941e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.07503777e-09 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.84129635e-12 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(2207, 25)
[[2.29605688e-10 1.19716023e-06 9.99808453e-01 ... 3.01882584e-09
  1.11990380e-09 8.32722816e-12]
 [6.05665250e-16 8.83330488e-13 1.00042258e-10 ... 1.65264708e-15
  5.05221739e-16 4.35425970e-18]
 [2.55945023e-06 5.02408636e-04 2.01523789e-06 ... 2.63465197e-06
  2.48257818e-08 3.91439948e-10]
 ...
 [1.02811941e-06 1.85141607e-05 1.49806966e-06 ... 2.05409957e-05
  9.83010

In [11]:
# GPR model
gp_train_accuracy = []
gp_test_accuracy = []
gp_r2s = []
gp_time = []
gpr_model = None

# From ts linear
for fold in range(number_of_folds):
    
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gpr_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, normalize_y=False)
    indices = np.random.choice(x_train.shape[0], 2000, replace=False)        

    start_time = time.time()
    gpr_model.fit(x_train[indices,:], y_train[indices,:])
    time_used = time.time() - start_time

    train_means = gpr_model.predict(x_train)
    test_means = gpr_model.predict(x_test)

    train_rmse = root_mean_squared_error(y_train, train_means)
    test_rmse = root_mean_squared_error(y_test, test_means)

    gp_train_accuracy.append(train_rmse)
    gp_test_accuracy.append(test_rmse)
    gp_r2 = r2_score(y_test, test_means)
    gp_r2s.append(gp_r2)
    gp_time.append(time_used)

gp_train_accuracy_mean = np.round(np.mean(gp_train_accuracy), 3)
gp_train_accuracy_std = np.round(np.std(gp_train_accuracy), 3)
gp_test_accuracy_mean = np.round(np.mean(gp_test_accuracy), 3)
gp_test_accuracy_std = np.round(np.std(gp_test_accuracy),3)
gp_r2_mean = np.round(np.mean(gp_r2s),3)
gp_r2_std = np.round(np.std(gp_r2s),3)
gp_time_mean = np.round(np.mean(gp_time),3)
gp_time_std = np.round(np.std(gp_time),3)

print(f"GP Train RMSE: {gp_train_accuracy_mean} ± {gp_train_accuracy_std}")
print(f"GP Test RMSE: {gp_test_accuracy_mean} ± {gp_test_accuracy_std}")
print(f"GP R2: {gp_r2_mean} ± {gp_r2_std}")
print(f"GP Time: {gp_time_mean} ± {gp_time_std}")


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-

GP Train RMSE: 554.697 ± 342.338
GP Test RMSE: 1697.938 ± 1086.378
GP R2: 0.09 ± 0.732
GP Time: 41.432 ± 67.67


In [12]:
# GPR model
gp_tsl_train_accuracy = []
gp_tsl_test_accuracy = []
gp_tsl_r2s = []
gp_tsl_time = []
gpr_tsl_model = None

# From ts linear
for fold in range(number_of_folds):
    
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gpr_tsl_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, normalize_y=False)
    tsl_model = get_tsl_model(x_train, y_train)

    start_time = time.time()
    gpr_tsl_model.fit(tsl_model.cen, ts_relu_wrapper(tsl_model.predict(tsl_model.cen + 1e-8)))
    time_used = time.time() - start_time

    train_means = gpr_tsl_model.predict(x_train)
    test_means = gpr_tsl_model.predict(x_test)

    train_rmse = root_mean_squared_error(y_train, train_means)
    test_rmse = root_mean_squared_error(y_test, test_means)

    gp_tsl_train_accuracy.append(train_rmse)
    gp_tsl_test_accuracy.append(test_rmse)
    gp_r2 = r2_score(y_test, test_means)
    gp_tsl_r2s.append(gp_r2)
    gp_tsl_time.append(time_used)

gp_train_accuracy_mean = np.round(np.mean(gp_tsl_train_accuracy), 3)
gp_train_accuracy_std = np.round(np.std(gp_tsl_train_accuracy), 3)
gp_test_accuracy_mean = np.round(np.mean(gp_tsl_test_accuracy), 3)
gp_test_accuracy_std = np.round(np.std(gp_tsl_test_accuracy),3)
gp_r2_mean = np.round(np.mean(gp_tsl_r2s),3)
gp_r2_std = np.round(np.std(gp_tsl_r2s),3)
gp_time_mean = np.round(np.mean(gp_tsl_time),3)
gp_time_std = np.round(np.std(gp_tsl_time),3)

print(f"GP Train RMSE: {gp_train_accuracy_mean} ± {gp_train_accuracy_std}")
print(f"GP Test RMSE: {gp_test_accuracy_mean} ± {gp_test_accuracy_std}")
print(f"GP R2: {gp_r2_mean} ± {gp_r2_std}")
print(f"GP Time: {gp_time_mean} ± {gp_time_std}")


FCM training RMSE: 269.16031836383513


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 270.78073730002455


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 269.5962942763495


...

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 271.1345745718356


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 269.1445279445027
GP Train RMSE: 389.545 ± 8.515
GP Test RMSE: 361.899 ± 14.105
GP R2: 0.971 ± 0.002
GP Time: 0.022 ± 0.007


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [13]:
# GPR model
gp_tsc_train_accuracy = []
gp_tsc_test_accuracy = []
gp_tsc_r2s = []
gp_tsc_time = []
gpr_tsc_model = None

# From ts constant
for fold in range(number_of_folds):
    
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gpr_tsc_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, normalize_y=False)

    start_time = time.time()
    tsc_model = get_tsc_model(x_train, y_train)
    gpr_tsc_model.fit(tsc_model.cen, ts_relu_wrapper(tsc_model.predict(tsc_model.cen + 1e-8)))
    time_used = time.time() - start_time

    train_means = gpr_tsc_model.predict(x_train)
    test_means = gpr_tsc_model.predict(x_test)

    train_rmse = root_mean_squared_error(y_train, train_means)
    test_rmse = root_mean_squared_error(y_test, test_means)

    gp_tsc_train_accuracy.append(train_rmse)
    gp_tsc_test_accuracy.append(test_rmse)
    gp_r2 = r2_score(y_test, test_means)
    gp_tsc_r2s.append(gp_r2)
    gp_tsc_time.append(time_used)

gp_train_accuracy_mean = np.round(np.mean(gp_tsc_train_accuracy), 3)
gp_train_accuracy_std = np.round(np.std(gp_tsc_train_accuracy), 3)
gp_test_accuracy_mean = np.round(np.mean(gp_tsc_test_accuracy), 3)
gp_test_accuracy_std = np.round(np.std(gp_tsc_test_accuracy),3)
gp_r2_mean = np.round(np.mean(gp_tsc_r2s),3)
gp_r2_std = np.round(np.std(gp_tsc_r2s),3)
gp_time_mean = np.round(np.mean(gp_tsc_time),3)
gp_time_std = np.round(np.std(gp_tsc_time),3)

print(f"GP Train RMSE: {gp_train_accuracy_mean} ± {gp_train_accuracy_std}")
print(f"GP Test RMSE: {gp_test_accuracy_mean} ± {gp_test_accuracy_std}")
print(f"GP R2: {gp_r2_mean} ± {gp_r2_std}")
print(f"GP Time: {gp_time_mean} ± {gp_time_std}")


(2207, 150)
[[3.53380486e-03 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.99999999e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.79946698e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [5.45859496e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.66175459e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.74977400e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(2207, 25)
[[3.53380486e-03 8.86815217e-06 9.88335472e-10 ... 1.20848028e-04
  9.68161143e-09 1.00464989e-08]
 [9.99999999e-01 1.00378774e-09 9.30241368e-17 ... 2.10223323e-12
  1.02537369e-15 4.78019622e-15]
 [2.79946698e-04 4.84314941e-05 1.21776777e-08 ... 1.72488626e-04
  1.90572733e-07 3.60945917e-05]
 ...
 [5.45859496e-06 1.15373657e-06 6.71675824e-06 ... 5.08752586e-05
  1.12259

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


(2207, 150)
[[2.26497851e-11 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.67400522e-17 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.75939041e-09 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.00638449e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.07972114e-10 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.10870977e-13 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(2207, 25)
[[2.26497851e-11 1.45807345e-07 4.61854035e-07 ... 1.90063815e-10
  1.25870358e-11 4.80436174e-10]
 [3.67400522e-17 1.00119868e-10 2.73525510e-14 ... 6.14439176e-16
  2.45474136e-17 2.77830622e-15]
 [3.75939041e-09 2.28611919e-06 1.06852633e-06 ... 2.90081853e-06
  7.33931647e-09 8.08945639e-08]
 ...
 [2.00638449e-06 1.83234893e-07 5.25260911e-06 ... 9.76875676e-07
  1.58412

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


(2207, 150)
[[1.73373953e-11 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.56817927e-17 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.55583272e-09 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [5.41937792e-07 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.89265950e-11 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.01416676e-13 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(2207, 25)
[[1.73373953e-11 1.46027378e-11 2.32100568e-10 ... 1.25698006e-09
  3.03266411e-09 5.53968731e-07]
 [2.56817927e-17 1.96039636e-17 6.26111358e-16 ... 3.50278595e-16
  1.69115745e-15 2.77179083e-14]
 [2.55583272e-09 6.86697394e-09 2.81762659e-06 ... 3.00981879e-09
  2.91608550e-06 1.14727065e-06]
 ...
 [5.41937792e-07 1.68802628e-07 1.03558594e-06 ... 4.63146589e-06
  1.97892

...

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


(2207, 150)
[[1.59952124e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.60555960e-13 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.36367969e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [6.54212986e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.03670983e-09 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.78269109e-09 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(2207, 25)
[[1.59952124e-06 2.66273669e-09 1.74494216e-04 ... 3.88141859e-09
  3.40431109e-09 3.00047411e-10]
 [1.60555960e-13 3.56238944e-16 1.00000000e+00 ... 1.29141336e-15
  7.22955588e-15 7.94601617e-17]
 [3.36367969e-05 3.13188039e-09 1.40925690e-05 ... 9.38217435e-09
  1.78032727e-07 9.11743212e-09]
 ...
 [6.54212986e-05 2.01055156e-05 3.74606859e-06 ... 4.62377506e-05
  1.18285

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


(2207, 150)
[[1.04674567e-07 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.09724388e-14 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.28889473e-08 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.12006405e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.21121632e-09 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.80057979e-08 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(2207, 25)
[[1.04674567e-07 9.92819143e-01 7.16449106e-07 ... 1.17037797e-04
  1.03188772e-05 5.49676757e-07]
 [2.09724388e-14 2.83661320e-11 9.98800721e-12 ... 3.35225014e-12
  1.69448811e-09 8.59037561e-13]
 [8.28889473e-08 4.65183749e-06 3.96136227e-03 ... 1.13004723e-04
  2.21491381e-05 1.62635338e-02]
 ...
 [1.12006405e-05 1.16156534e-06 1.12945942e-06 ... 1.00394925e-05
  1.94638

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [15]:
# GPR model
gp_nn_tsl_train_accuracy = []
gp_nn_tsl_test_accuracy = []
gp_nn_tsl_r2s = []
nn_tsl_r2s = []
gp_nn_tsl_time = []
gpr_nn_tsl_model = None

# From NN and ts linear
for fold in range(number_of_folds):
    
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gpr_nn_tsl_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, normalize_y=False)
    tsl_model = get_tsl_model(x_train, y_train)

    start_time = time.time()
    inputs = tsl_model.cen
    ts_predictions = ts_relu_wrapper(tsl_model.predict(tsl_model.cen + 1e-8))
    nn_predictions = nn_model.predict(inputs)
    gpr_nn_tsl_model.fit(inputs, nn_predictions)
    time_used = time.time() - start_time

    train_means = gpr_nn_tsl_model.predict(x_train)
    test_means = gpr_nn_tsl_model.predict(x_test)

    train_rmse = root_mean_squared_error(y_train, train_means)
    test_rmse = root_mean_squared_error(y_test, test_means)

    gp_nn_tsl_train_accuracy.append(train_rmse)
    gp_nn_tsl_test_accuracy.append(test_rmse)
    gp_r2 = r2_score(y_test, test_means)
    gp_nn_tsl_r2s.append(gp_r2)
    gp_nn_tsl_time.append(time_used)
    nn_tsl_r2s.append(r2_score(ts_predictions, nn_predictions))

gp_train_accuracy_mean = np.round(np.mean(gp_nn_tsl_train_accuracy), 3)
gp_train_accuracy_std = np.round(np.std(gp_nn_tsl_train_accuracy), 3)
gp_test_accuracy_mean = np.round(np.mean(gp_nn_tsl_test_accuracy), 3)
gp_test_accuracy_std = np.round(np.std(gp_nn_tsl_test_accuracy),3)
gp_r2_mean = np.round(np.mean(gp_nn_tsl_r2s),3)
gp_r2_std = np.round(np.std(gp_nn_tsl_r2s),3)
gp_time_mean = np.round(np.mean(gp_nn_tsl_time),3)
gp_time_std = np.round(np.std(gp_nn_tsl_time),3)
nn_ts_r2_mean = np.round(np.mean(nn_tsl_r2s),3)
nn_ts_r2_std = np.round(np.std(nn_tsl_r2s),3)

print(f"GP Train RMSE: {gp_train_accuracy_mean} ± {gp_train_accuracy_std}")
print(f"GP Test RMSE: {gp_test_accuracy_mean} ± {gp_test_accuracy_std}")
print(f"GP R2: {gp_r2_mean} ± {gp_r2_std}")
print(f"GP Time: {gp_time_mean} ± {gp_time_std}")
print(f"TS NN R2: {nn_ts_r2_mean} ± {nn_ts_r2_std}")


FCM training RMSE: 267.8068950158151
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 266.3864916215174
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 271.2994731957097
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


...

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 269.42275050286185
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 269.23290088154243
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step
GP Train RMSE: 589.613 ± 36.378
GP Test RMSE: 543.753 ± 35.96
GP R2: 0.933 ± 0.009
GP Time: 0.034 ± 0.009
TS NN R2: 0.967 ± 0.006


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
